In [3]:
import tensorflow as tf, numpy as np
import os, librosa
from PIL import Image

In [2]:
drum_path = './Loop/'

drums = os.listdir(drum_path)

In [11]:
count = 0

for drum in drums:
    y, sr = librosa.load(drum_path+drum)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    np.save(), mel)
    count += 1
    

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cid2r\\Documents\\realVSCode\\AutoEncoder_Drummer\\drum_npy\\0.npy'